# Orderbook 5000 V 3.0

### This script fetches order book data from the Binance API, creates a Plotly line chart to visualize the top bid and ask prices, and saves the top 5 highest total value prices for both bids and asks into a DataFrame

In [1]:
import plotly.graph_objects as go
import pandas as pd
import requests
import time
from datetime import datetime
import json

pd.options.display.float_format = '{:.2f}'.format

def get_order_book(symbol, limit):
    url = f"https://api.binance.com/api/v3/depth"
    params = {"symbol": symbol, "limit": limit}
    response = requests.get(url, params=params)
    data = response.json()
    return data

def create_chart(bids, asks, limit, hdf):
    bids_prices = [float(bid[0]) for bid in bids]
    bids_quantities = [float(bid[1]) for bid in bids]
    asks_prices = [float(ask[0]) for ask in asks]
    asks_quantities = [float(ask[1]) for ask in asks]

    # Calculate total value (amount) at each price level
    bids_values = [price * quantity for price, quantity in zip(bids_prices, bids_quantities)]
    asks_values = [price * quantity for price, quantity in zip(asks_prices, asks_quantities)]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=bids_prices, y=bids_values, mode='lines', name='Bids'))
    fig.add_trace(go.Scatter(x=asks_prices, y=asks_values, mode='lines', name='Asks'))

    fig.update_xaxes(title_text="Price")
    fig.update_yaxes(title_text="Total Value")

    current_datetime = time.strftime("%Y-%m-%d %H:%M")
    fig.update_layout(
        title=f'Order Book Line Chart - Limit: {limit} - {current_datetime}',
        showlegend=True,
        height=1000,  # adjust height as needed
        width=1400    # adjust width as needed
    )

    fig.show()

    # Create datetime index
    datetime_index = pd.to_datetime([current_datetime] * 5)

    # Extract top 5 highest total value prices for bids and asks
    top_5_bids = sorted(zip(bids_prices, bids_values), key=lambda x: x[1], reverse=True)[:5]
    top_5_asks = sorted(zip(asks_prices, asks_values), key=lambda x: x[1], reverse=True)[:5]

    # Convert to DataFrames and concatenate
    top_5_bids_df = pd.DataFrame(top_5_bids, columns=['Price', 'Total Value (Bids)'], index=datetime_index)
    top_5_asks_df = pd.DataFrame(top_5_asks, columns=['Price', 'Total Value (Asks)'], index=datetime_index)

    hdf = pd.concat([hdf, top_5_bids_df, top_5_asks_df], axis=1)

    return hdf


symbol = 'BTCUSDT'
limit_values = [5000]  # Modify this list with your desired limit values
hdf = pd.DataFrame()
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M")

order_book = get_order_book(symbol, limit_values)
bids = order_book["bids"]
asks = order_book["asks"]
hdf = create_chart(bids, asks, limit_values, hdf)

hdf['Total Value (Bids)'] = hdf['Total Value (Bids)'] / 1000000
hdf['Total Value (Asks)'] = hdf['Total Value (Asks)'] / 1000000

print(hdf)  # You can print hdf to see the dataframe with top 5 highest total value prices for both bids and asks
hdf.to_csv(f"{current_datetime}.csv", index=True)
# Send to Telegram




                       Price  Total Value (Bids)    Price  Total Value (Asks)
2024-04-09 03:25:00 71233.10                0.77 72400.00                2.34
2024-04-09 03:25:00 71385.90                0.46 72000.00                1.23
2024-04-09 03:25:00 71550.00                0.42 71744.14                0.80
2024-04-09 03:25:00 71480.00                0.40 72078.00                0.78
2024-04-09 03:25:00 71481.20                0.37 71622.50                0.63


#### This part is responsible for sending a message containing important levels extracted from the DataFrame hdf to a Telegram chat.

In [ ]:
TOKEN = '6624530085:AAFp2tFnN1YzuK3A8g2Ym-AKItefItOPDXY'
chat_id = '@plwerdfgg2'
message = f"Important Levels By Orderbook5000:\n{hdf}"
urltelegram = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"
print(requests.get(urltelegram).json())